In [1]:
import pandas
import torch
import torch.nn as nn 
import torch.optim as optim
import torch.onnx
import numpy

In [2]:
# Load the dataset into a dataframe
dataframe = pandas.read_csv("../datasets/maternal_health_risk.csv")

In [3]:
dataframe = pandas.get_dummies(dataframe, columns=['RiskLevel'])

In [4]:
print(dataframe.iloc[[0],:6]) # inputs

   Age  SystolicBP  DiastolicBP    BS  BodyTemp  HeartRate
0   25         130           80  15.0      98.0         86


In [5]:
print(dataframe.iloc[[0],6:]) # outputs

   RiskLevel_high risk  RiskLevel_low risk  RiskLevel_mid risk
0                 True               False               False


In [6]:
# split dataset to train (70%) and test (30%)
train, test = numpy.split(dataframe.to_numpy(dtype="float32"), [int(.7*len(dataframe))])

In [7]:
# further split training and testing to their corresponding inputs and outputs
train_torch = torch.from_numpy(train)
test_torch = torch.from_numpy(test)

train_x = train_torch[:,:6]
train_y = train_torch[:,6:]
test_x = test_torch[:,:6]
test_y = test_torch[:,6:]

In [8]:
print(train_x[0])
print(train_y[0])

tensor([ 25., 130.,  80.,  15.,  98.,  86.])
tensor([1., 0., 0.])


In [9]:
model = nn.Sequential(
    nn.Linear(6, 12),
    nn.ReLU(),
    nn.Linear(12, 6),
    nn.ReLU(),
    nn.Linear(6, 3)
)

In [10]:
# loss function and optimizer
loss_function = nn.MSELoss()  # mean square error
optimizer = optim.Adam(model.parameters(), lr=0.00001)

In [11]:
# training parameters
n_epochs = 20000   # number of epochs to run

In [12]:
for index in range(n_epochs):
    prediction_y = model(train_x)
    step_loss = loss_function(prediction_y, train_y)
    optimizer.zero_grad()
    step_loss.backward()
    optimizer.step()
    print ('epoch [{}], Loss: {:.2f}'.format(index, step_loss.item()))

epoch [0], Loss: 104.18
epoch [1], Loss: 104.11
epoch [2], Loss: 104.04
epoch [3], Loss: 103.98
epoch [4], Loss: 103.91
epoch [5], Loss: 103.85
epoch [6], Loss: 103.78
epoch [7], Loss: 103.72
epoch [8], Loss: 103.65
epoch [9], Loss: 103.59
epoch [10], Loss: 103.52
epoch [11], Loss: 103.46
epoch [12], Loss: 103.39
epoch [13], Loss: 103.33
epoch [14], Loss: 103.26
epoch [15], Loss: 103.20
epoch [16], Loss: 103.13
epoch [17], Loss: 103.07
epoch [18], Loss: 103.00
epoch [19], Loss: 102.94
epoch [20], Loss: 102.87
epoch [21], Loss: 102.81
epoch [22], Loss: 102.74
epoch [23], Loss: 102.68
epoch [24], Loss: 102.61
epoch [25], Loss: 102.55
epoch [26], Loss: 102.48
epoch [27], Loss: 102.42
epoch [28], Loss: 102.36
epoch [29], Loss: 102.29
epoch [30], Loss: 102.23
epoch [31], Loss: 102.16
epoch [32], Loss: 102.10
epoch [33], Loss: 102.03
epoch [34], Loss: 101.97
epoch [35], Loss: 101.91
epoch [36], Loss: 101.84
epoch [37], Loss: 101.78
epoch [38], Loss: 101.71
epoch [39], Loss: 101.65
epoch [40]

In [13]:
prediction_y = model(test_x)
print(prediction_y)
print(test_y)

tensor([[ 0.1496,  0.3766,  0.2582],
        [ 0.6913,  0.3502,  0.1144],
        [ 0.2562,  0.3714,  0.2299],
        [ 0.1496,  0.3766,  0.2582],
        [ 0.1496,  0.3766,  0.2582],
        [ 0.1496,  0.3766,  0.2582],
        [ 0.3520,  0.3668,  0.2045],
        [ 0.1496,  0.3766,  0.2582],
        [ 0.1496,  0.3766,  0.2582],
        [ 0.1496,  0.3766,  0.2582],
        [ 0.2052,  0.3739,  0.2434],
        [ 0.1496,  0.3766,  0.2582],
        [ 0.1496,  0.3766,  0.2582],
        [ 0.7190,  0.3489,  0.1071],
        [ 0.1818,  0.3750,  0.2496],
        [ 0.1496,  0.3766,  0.2582],
        [ 0.1961,  0.3744,  0.2458],
        [ 0.1496,  0.3766,  0.2582],
        [ 0.1496,  0.3766,  0.2582],
        [ 0.1496,  0.3766,  0.2582],
        [ 0.1496,  0.3766,  0.2582],
        [ 0.1496,  0.3766,  0.2582],
        [ 0.6913,  0.3502,  0.1144],
        [ 0.2562,  0.3714,  0.2299],
        [ 0.1496,  0.3766,  0.2582],
        [ 0.1496,  0.3766,  0.2582],
        [ 0.1496,  0.3766,  0.2582],
 

In [14]:
# Accuracy when matching the interpretation of inputs and outputs
prediction_boolean = prediction_y.detach().numpy() > 0.5
test_boolean = test_y.numpy() > 0.5
correct_boolean = (prediction_boolean == test_boolean).sum()
total_boolean = test_boolean.size
print('Match: {}, Total: {}, Rate: {:.2f}%'.format(correct_boolean, total_boolean, correct_boolean/total_boolean * 100))

Match: 616, Total: 915, Rate: 67.32%


In [15]:
# Accuracy when EXACT label basis
prediction_labeled = numpy.argmax(prediction_y.detach().numpy(), axis=-1)
test_labeled = numpy.argmax(test_y.numpy(), axis=-1)
correct_answers = (prediction_labeled == test_labeled).sum()
total = test_labeled.size
print('Exact: {}, Total: {}, Rate: {:.2f}%'.format(correct_answers, total, correct_answers/total * 100))

Exact: 92, Total: 305, Rate: 30.16%


In [16]:
model.eval()
torch.onnx.export(model, test_x[0], "../onnx/maternal_health_risk_model.onnx", export_params=True)